In [ ]:
# 데이터 처리 및 분석을 위한 기본 라이브러리
import numpy as np
import pandas as pd

# 데이터 시각화 도구
import matplotlib.pyplot as plt
import seaborn as sns

# 이미지 처리 관련 라이브러리
from PIL import Image
from glob import glob

# 진행 상태를 추적하는 데 도움이 되는 라이브러리
from tqdm import tqdm

# 딥러닝 모델링을 위한 TensorFlow 및 관련 콜백
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# 다운로드가 안 될 경우 아래 명령어의 주석을 해제하고 실행한 후 다시 시도해보세요.
# !pip install --upgrade --no-cache-dir gdown

In [ ]:
!gdown 12SmqZSWY8IGyPvCgdgpR7SXUQzSunCDE
!unzip cityscapes.zip

In [ ]:
import os

def get_image_paths(directory):
    return glob(os.path.join(directory, '*.jpg'))

train_dir = 'cityscapes/train/'
val_dir = 'cityscapes/val/'

train_image_paths = get_image_paths(train_dir)
val_image_paths = get_image_paths(val_dir)

print(f"Number of training images: {len(train_image_paths)}")
print(f"Number of validation images: {len(val_image_paths)}")

In [ ]:
from collections import namedtuple
Label = namedtuple( 'Label' , ['name', 'id' ,'trainId','category','categoryId','hasInstances','ignoreInEval','color'] )

labels=[
Label('unlabeled',0,255,'void',0,False,True,(0,0,0)),
Label('egovehicle',1,255,'void',0,False,True,(0,0,0)),
Label('rectificationborder',2,255,'void',0,False,True,(0,0,0)),
Label('outofroi',3,255,'void',0,False,True,(0,0,0)),
Label('static',4,255,'void',0,False,True,(0,0,0)),
Label('dynamic',5,255,'void',0,False,True,(111,74,0)),
Label('ground',6,255,'void',0,False,True,(81,0,81)),
Label('road',7,0,'ground',1,False,False,(128,64,128)),
Label('sidewalk',8,1,'ground',1,False,False,(244,35,232)),
Label('parking',9,255,'ground',1,False,True,(250,170,160)),
Label('railtrack',10,255,'ground',1,False,True,(230,150,140)),
Label('building',11,2,'construction',2,False,False,(70,70,70)),
Label('wall',12,3,'construction',2,False,False,(102,102,156)),
Label('fence',13,4,'construction',2,False,False,(190,153,153)),
Label('guardrail',14,255,'construction',2,False,True,(180,165,180)),
Label('bridge',15,255,'construction',2,False,True,(150,100,100)),
Label('tunnel',16,255,'construction',2,False,True,(150,120,90)),
Label('pole',17,5,'object',3,False,False,(153,153,153)),
Label('polegroup',18,255,'object',3,False,True,(153,153,153)),
Label('trafficlight',19,6,'object',3,False,False,(250,170,30)),
Label('trafficsign',20,7,'object',3,False,False,(220,220,0)),
Label('vegetation',21,8,'nature',4,False,False,(107,142,35)),
Label('terrain',22,9,'nature',4,False,False,(152,251,152)),
Label('sky',23,10,'sky',5,False,False,(70,130,180)),
Label('person',24,11,'human',6,True,False,(220,20,60)),
Label('rider',25,12,'human',6,True,False,(255,0,0)),
Label('car',26,13,'vehicle',7,True,False,(0,0,142)),
Label('truck',27,14,'vehicle',7,True,False,(0,0,70)),
Label('bus',28,15,'vehicle',7,True,False,(0,60,100)),
Label('caravan',29,255,'vehicle',7,True,True,(0,0,90)),
Label('trailer',30,255,'vehicle',7,True,True,(0,0,110)),
Label('train',31,16,'vehicle',7,True,False,(0,80,100)),
Label('motorcycle',32,17,'vehicle',7,True,False,(0,0,230)),
Label('bicycle',33,18,'vehicle',7,True,False,(119,11,32)),
Label('licenseplate',34,19,'vehicle',7,False,True,(0,0,142)),
]

In [ ]:
N_FILTERS = 64
KERNEL_SIZE = 3
N_CLASSES = len(labels)
IMAGE_SIZE = 128
EPOCHS = 15
BATCH_SIZE = 16

MODEL_CHECKPOINT_FILEPATH = './cityscapes-unet.ckpt'

id2color = { label.id : np.asarray(label.color) for label in labels } #id에 따른 컬러

In [ ]:
def image_mask_split(filename, image_size):
    image_mask = Image.open(filename)
    image, mask = image_mask.crop([0, 0, 256, 256]), image_mask.crop([256, 0, 512, 256])
    image = image.resize((image_size, image_size))
    mask = mask.resize((image_size, image_size))
    image = np.array(image) / 255
    mask = np.array(mask)

    return image, mask

def find_closest_labels_vectorized(mask, mapping):
    closest_distance = np.full([mask.shape[0], mask.shape[1]], 10000)
    closest_category = np.full([mask.shape[0], mask.shape[1]], None)
    for id, color in mapping.items():
        dist = np.sqrt(np.linalg.norm(mask - color.reshape([1,1,-1]), axis=-1))
        is_closer = closest_distance > dist
        closest_distance = np.where(is_closer, dist, closest_distance)
        closest_category = np.where(is_closer, id, closest_category)
    return closest_category

In [ ]:
train_images = []
train_masks = []
train_masks_enc = []
val_images = []
val_masks = []
val_masks_enc = []

train_lists = glob(f'{train_dir}*.jpg')
val_lists = glob(f'{val_dir}*.jpg')
print(len(train_lists), len(val_lists))

for train_file in tqdm(train_lists, desc = 'Building Training Dataset: '):
    image, mask = image_mask_split(train_file, IMAGE_SIZE)
    train_images.append(image)
    train_masks.append(mask)
    train_masks_enc.append(find_closest_labels_vectorized(mask, id2color))

for val_file in tqdm(val_lists, desc = 'Building Validation Dataset: '):
    image, mask = image_mask_split(val_file, IMAGE_SIZE)
    val_images.append(image)
    val_masks.append(mask)
    val_masks_enc.append(find_closest_labels_vectorized(mask, id2color))

In [ ]:
plt.figure(figsize=[20, 14])


for i in range(2):
    img = train_images[i]
    msk = train_masks[i]
    enc = train_masks_enc[i]

    tmp = np.zeros([enc.shape[0], enc.shape[1], 3])
    for row in range(enc.shape[0]):
        for col in range(enc.shape[1]):
            tmp[row, col, :] = id2color[enc[row, col]]
            tmp = tmp.astype('uint8')

    plt.subplot(2, 3, i*3 + 1)
    plt.imshow(img)
    plt.axis('off')
    plt.gca().set_title('Sample image {}'.format(str(i+1)))

    plt.subplot(2, 3, i*3 + 2)
    plt.imshow(msk)
    plt.axis('off')
    plt.gca().set_title('Sample mask {}'.format(str(i+1)))

    plt.subplot(2, 3, i*3 + 3)
    plt.imshow(tmp)
    plt.axis('off')
    plt.gca().set_title('Encoded sameple mask {}'.format(str(i+1)))

plt.subplots_adjust(wspace=0, hspace=0.1)

In [ ]:
# 이미지와 마스크를 numpy 배열로 변환하고 데이터 타입을 'float32'로 설정
train_images = np.stack(train_images).astype('float32')
train_masks_enc = np.stack(train_masks_enc).astype('float32')

val_images = np.stack(val_images).astype('float32')
val_masks_enc = np.stack(val_masks_enc).astype('float32')

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size=3):  # conv2d_block 함수를 정의합니다.
    x = input_tensor  # 입력 텐서를 변수 x에 저장합니다.
    for i in range(2):  # 2개의 Conv2D 레이어를 추가하기 위해 반복합니다.
        # Conv2D 레이어를 추가합니다. He 초기화와 ReLU 활성화를 사용하고, 패딩은 'same'으로 설정합니다.
        x = tf.keras.layers.Conv2D(filters=n_filters, kernel_size=kernel_size,
                                   kernel_initializer='he_normal', activation='relu', padding='same')(x)
    return x  # 마지막 레이어의 출력을 반환합니다.


def encoder_block(inputs, n_filters=64, pool_size=(2,2), dropout=0.3):  # encoder_block 함수를 정의합니다.
    f = conv2d_block(inputs, n_filters)  # 입력에 대해 conv2d_block을 적용하여 특징 맵을 생성합니다.
    p = tf.keras.layers.MaxPooling2D(pool_size=pool_size)(f)  # MaxPooling2D 레이어를 적용하여 다운샘플링합니다.
    p = tf.keras.layers.Dropout(dropout)(p)  # 드롭아웃을 적용합니다.

    return f, p  # 특징 맵과 풀링된 출력을 반환합니다.


def encoder(inputs):  # encoder 함수를 정의합니다.
    f1, p1 = encoder_block(inputs, n_filters=64)  # 첫 번째 인코더 블록을 추가합니다.
    f2, p2 = encoder_block(p1, n_filters=128)  # 두 번째 인코더 블록을 추가합니다.
    f3, p3 = encoder_block(p2, n_filters=256)  # 세 번째 인코더 블록을 추가합니다.
    f4, p4 = encoder_block(p3, n_filters=512)  # 네 번째 인코더 블록을 추가합니다.

    return p4, (f1, f2, f3, f4)  # 마지막 풀링된 출력과 모든 특징 맵을 반환합니다.

def bottleneck(inputs):  # bottleneck 함수를 정의합니다.
    bottle_neck = conv2d_block(inputs, n_filters=1024)  # conv2d_block을 적용하여 병목 레이어를 생성합니다.
    return bottle_neck  # 병목 레이어의 출력을 반환합니다.

# Decoder
def decoder_block(inputs, conv_output, n_filters=64, kernel_size=3, strides=3, dropout=0.3):  # decoder_block 함수를 정의합니다.
    u = tf.keras.layers.Conv2DTranspose(n_filters, kernel_size, strides, padding='same')(inputs)  # Conv2DTranspose 레이어를 적용하여 업샘플링합니다.
    c = tf.keras.layers.concatenate([u, conv_output])  # 업샘플링된 출력과 인코더의 출력을 연결합니다.
    c = tf.keras.layers.Dropout(dropout)(c)  # 드롭아웃을 적용합니다.
    c = conv2d_block(c, n_filters)  # conv2d_block을 적용하여 특징 맵을 생성합니다.

    return c  # 마지막 레이어의 출력을 반환합니다.

def decoder(inputs, convs, output_channels):  # decoder 함수를 정의합니다.
    f1, f2, f3, f4 = convs  # 인코더에서 전달된 특징 맵들을 분해합니다.

    c6 = decoder_block(inputs, f4, n_filters=512, kernel_size=3, strides=2)  # 첫 번째 디코더 블록을 추가합니다.
    c7 = decoder_block(c6, f3, n_filters=256, kernel_size=3, strides=2)  # 두 번째 디코더 블록을 추가합니다.
    c8 = decoder_block(c7, f2, n_filters=128, kernel_size=3, strides=2)  # 세 번째 디코더 블록을 추가합니다.
    c9 = decoder_block(c8, f1, n_filters=64, kernel_size=3, strides=2)  # 네 번째 디코더 블록을 추가합니다.
    outputs = tf.keras.layers.Conv2D(output_channels, 1, activation='softmax')(c9)  # 마지막 Conv2D 레이어를 추가하여 출력 채널 수를 맞추고 softmax 활성화를 적용합니다.

    return outputs  # 최종 출력을 반환합니다.

In [ ]:
OUTPUT_CHANNELS = N_CLASSES

def UNet():
    inputs = tf.keras.layers.Input(shape=(IMAGE_SIZE,IMAGE_SIZE,3))

    encoder_output, convs = encoder(inputs)
    bottle_neck = bottleneck(encoder_output)
    outputs = decoder(bottle_neck, convs, OUTPUT_CHANNELS)

    model = tf.keras.Model(inputs, outputs)

    return model

model = UNet()
model.summary()

tf.keras.utils.plot_model(model, show_shapes = True,dpi=150)

In [ ]:
model.compile(optimizer = 'adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics = ['accuracy'])
model_checkpoint = ModelCheckpoint(MODEL_CHECKPOINT_FILEPATH, monitor='val_loss', save_best_only=True, save_weights_only=True,verbose=1, mode = 'min')
history = model.fit(x = train_images,
                    y = train_masks_enc,
                    batch_size = BATCH_SIZE,
                    epochs = EPOCHS,
                    validation_data = (val_images, val_masks_enc),
                    callbacks = [model_checkpoint])

In [ ]:
# 모델 시각화
plt.plot(history.history['loss'], label='Train loss')
plt.plot(history.history['val_loss'], label = 'Validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='lower right')

In [ ]:
plt.figure(figsize=[15, 20])

for i in range(4):
    img = val_images[i] # i번째 이미지를 가져옵니다.
    enc = val_masks_enc[i] # i번째 이미지에 대한 정답 마스크를 가져옵니다.

    # 이미지를 모델에 전달하여 예측을 생성합니다.
    # reshape 함수를 사용하여 입력을 모델이 예상하는 형태로 만듭니다.
    pred = model.predict(img.reshape([1] + [IMAGE_SIZE, IMAGE_SIZE, 3]))
    # 예측 결과는 각 클래스에 대한 확률을 포함하므로 argmax 함수를 사용하여 가장 높은 확률을 가진 클래스를 선택합니다.
    pred = np.squeeze(np.argmax(pred, axis=-1))

    # 마스크와 예측을 시각화할 빈 이미지를 생성합니다.
    tmp1 = np.zeros([enc.shape[0], enc.shape[1], 3])
    tmp2 = np.zeros([enc.shape[0], enc.shape[1], 3])

    # 각 픽셀에 대해 반복합니다.
    for row in range(enc.shape[0]):
        for col in range(enc.shape[1]):
            # 각 픽셀의 레이블에 해당하는 색을 id2color 사전에서 찾아서 임시 이미지에 할당합니다.
            tmp1[row, col, :] = id2color[enc[row, col]]
            tmp1 = tmp1.astype('uint8') # uint8 형식으로 변환합니다.

            tmp2[row, col, :] = id2color[pred[row, col]]
            tmp2 = tmp2.astype('uint8') # uint8 형식으로 변환합니다.

    # 원본 이미지, 마스크, 예측 결과를 그림에 그립니다.
    plt.subplot(4, 3, i*3 + 1)
    plt.imshow(img)
    plt.axis('off')
    plt.gca().set_title('Image {}'.format(str(i+1)))

    plt.subplot(4, 3, i*3 + 2)
    plt.imshow(tmp1)
    plt.axis('off')
    plt.gca().set_title('Encoded Mask {}'.format(str(i+1)))

    plt.subplot(4, 3, i*3 + 3)
    plt.imshow(tmp2)
    plt.axis('off')
    plt.gca().set_title('Model Prediction {}'.format(str(i+1)))

# subplot 사이의 간격을 조정합니다.
plt.subplots_adjust(wspace=0, hspace=0.1)